In [ ]:
!pip install pyspark

### Matriz de Admitância \(Y_{bus}\)

Imagine um sistema elétrico complexo com milhares de quilômetros de linhas. Para que um computador consiga representar como a energia flui, precisamos de um modelo matemático em que cada linha e cada transformador ocupem um lugar bem definido.

A matriz de admitância nodal, chamada de **matriz \(Y_{bus}\)**, é exatamente essa representação: uma matriz quadrada de dimensão \(n \times n\), onde \(n\) é o número de barras (nós) do sistema elétrico.

---

### Base matemática

A construção da matriz \(Y_{bus}\) se apoia em duas leis fundamentais:

- **Lei de Kirchhoff das Correntes (LKC)** aplicada aos nós da rede;
- **Lei de Ohm** escrita em forma fasorial.

A relação básica entre correntes injetadas, tensões nodais e admitâncias é dada por:

\[
\mathbf{I} = \mathbf{Y}_{\text{bus}} \, \mathbf{V}
\]

Onde:

- \(\mathbf{I}\): vetor das correntes injetadas em cada barra;
- \(\mathbf{V}\): vetor das tensões em cada barra;
- \(\mathbf{Y}_{\text{bus}}\): matriz de admitância nodal (com \(Y = 1/Z\), o inverso da impedância).

---

### Regras de formação da matriz \(Y_{bus}\)

Para montar \(\mathbf{Y}_{\text{bus}}\), utilizamos duas regras principais para cada elemento da matriz:

- **Elementos da diagonal principal \(Y_{ii}\)**  
  Representam a soma de todas as admitâncias conectadas diretamente ao nó \(i\), incluindo as admitâncias shunt (capacitâncias de linha, derivação de compensadores, etc.):

  \[
  Y_{ii} = \sum_{j=1}^{n} y_{ij}
  \]

- **Elementos fora da diagonal \(Y_{ij}\)**  
  Representam o negativo da admitância equivalente da ligação direta entre as barras \(i\) e \(j\). Se não houver linha ou transformador ligando diretamente \(i\) a \(j\), o elemento é nulo:

  \[
  Y_{ij} = -\,y_{ij}
  \]

---

### Importância da matriz \(Y_{bus}\)

A matriz \(Y_{bus}\) é a **espinha dorsal** de diversos algoritmos clássicos em sistemas de potência, por exemplo:

- **Fluxo de potência**: avaliação de sobrecarga em linhas e perfis de tensão;
- **Cálculo de curto-circuito**: dimensionamento de disjuntores e ajustes de proteção;
- **Estudos de estabilidade**: análise da resposta do sistema após perturbações e falhas.


### Exemplos de sistemas (3 barras, IEEE 14 e IEEE 30)

A modelagem de sistemas como o de **3 barras**, o **IEEE 14** e o **IEEE 30** utiliza a mesma base matemática da matriz de admitância \(Y_{bus}\). O que muda é apenas:

- a **dimensão** da matriz (número de barras \(N\));
- a **complexidade** e a **quantidade** de dados de entrada (\(R\), \(X\) e \(B\) das linhas e transformadores).

À medida que o sistema cresce (por exemplo, o caso IEEE 30), a matriz \(Y_{bus}\) deixa de ser algo tratável manualmente e passa a ser uma **matriz esparsa** (a maior parte dos elementos é zero).

---

### Passo 1: Cálculo das admitâncias de linha e shunt

Para qualquer um desses sistemas, o primeiro passo é converter os dados de resistência, reatância e susceptância shunt em admitâncias:

- **Admitância de série** entre as barras \(i\) e \(j\):

  \[
  y_{ij} = \frac{1}{R_{ij} + j X_{ij}}
  \]

- **Admitância shunt** da linha (normalmente distribuída nas duas extremidades):

  \[
  y_{\text{sh}} = j \, \frac{B_{ij}}{2}
  \]

Essa admitância shunt é adicionada às barras terminais da linha.

---

### Passo 2: Elementos da diagonal \(Y_{ii}\)

Para cada barra \(i\), o elemento diagonal da matriz \(Y_{bus}\) é obtido somando todas as admitâncias conectadas a ela:

\[
Y_{ii} = \sum_{j \in \Omega_i} y_{ij} 
       + \sum_{j \in \Omega_i} y_{\text{sh},ij}
       + y_{\text{carga},i}
\]

Onde \(\Omega_i\) é o conjunto de barras conectadas à barra \(i\).

Os tamanhos típicos da matriz são:

- Sistema 3 barras: matriz \(3 \times 3\);
- IEEE 14: matriz \(14 \times 14\);
- IEEE 30: matriz \(30 \times 30\).

---

### Passo 3: Elementos fora da diagonal \(Y_{ij}\)

Os elementos que conectam as barras \(i\) e \(j\) (com \(i \neq j\)) são dados por:

\[
Y_{ij} = Y_{ji} = -\,y_{ij}
\]

Se **não** existir linha ou transformador entre \(i\) e \(j\), então:

\[
Y_{ij} = 0
\]

Isso explica por que, em sistemas como o IEEE 30, a matriz \(Y_{bus}\) é esparsa: cada barra se conecta diretamente a poucas vizinhas.

---

### Dimensão e interpretação física

De forma geral, para um sistema com \(N\) barras, a matriz de admitância nodal tem dimensão:

\[
\mathbf{Y}_{\text{bus}} \in \mathbb{C}^{N \times N}
\]

E a relação fundamental entre correntes injetadas e tensões nodais permanece a mesma para todos os casos (3 barras, IEEE 14, IEEE 30, etc.):

\[
\mathbf{I} = \mathbf{Y}_{\text{bus}} \, \mathbf{V}
\]

Essa equação é a base tanto para **fluxo de potência** quanto para **cálculos de curto-circuito** e outros estudos em sistemas elétricos de potência.


## 1. Extração e SQL com PySpark

O PySpark é ideal para carregar grandes volumes de dados de sistemas (como o IEEE 118 ou maiores) e realizar filtros via SQL.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("AnaliseSistemas").getOrCreate()

# Simulando dados que viriam do deck .pwf (DLIN)
data_linhas = [
    (1, 2, 0.02, 0.06, 0.03), # De, Para, R, X, B/2
    (2, 3, 0.01, 0.03, 0.01),
    (1, 3, 0.01, 0.05, 0.02)
]

df_linhas = spark.createDataFrame(data_linhas, ["DE", "PARA", "R", "X", "B_MEIO"])
df_linhas.createOrReplaceTempView("linhas")

# SQL para calcular a admitância de série (y = 1/z)
# y = (R - jX) / (R² + X²)
query = """
SELECT DE, PARA, 
       (R / (R*R + X*X)) as G, 
       (-X / (R*R + X*X)) as B,
       B_MEIO
FROM linhas
"""
df_calculado = spark.sql(query)


ModuleNotFoundError: No module named 'pyspark'

## 2. Integração com SymPy para a Matriz Analítica

Para sistemas como o de 3 barras, o uso do SymPy permite manter as variáveis como símbolos, útil para sensibilidade de tensão. Para os casos IEEE 14 e 30, convertemos os resultados do Spark para Pandas antes de alimentar a matriz.

In [ ]:
import sympy as sp
import pandas as pd

# Convertendo Spark para Pandas (apenas para o processamento da matriz)
pdf = df_calculado.toPandas()
n = 3 # Exemplo 3 barras
Ybus = sp.zeros(n, n)

for index, row in pdf.iterrows():
    i, j = int(row['DE'])-1, int(row['PARA'])-1
    y_serie = complex(row['G'], row['B'])
    y_shunt = complex(0, row['B_MEIO'])
    
    # Elementos fora da diagonal
    Ybus[i, j] -= y_serie
    Ybus[j, i] -= y_serie
    
    # Elementos da diagonal
    Ybus[i, i] += y_serie + y_shunt
    Ybus[j, j] += y_serie + y_shunt

# Visualização simbólica
sp.pprint(Ybus)


## 3. Conversão PWF para PySpark (Leitura de Dados)

Como o arquivo .pwf não é CSV, usamos o Pandas para ler as larguras fixas das seções DBAR e DLIN e depois enviamos ao Spark para consultas SQL.


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
import pandapower as pp
import pandapower.converter as pc

spark = SparkSession.builder.getOrCreate()

def pwf_to_spark(file_path, section="DLIN"):
    # Exemplo simplificado de colunas DLIN (De, Para, R, X, B...)
    # No .pwf real, as posições são rígidas (ex: 1-5, 6-10...)
    widths = [5, 5, 2, 6, 6, 6] 
    cols = ["DE", "PARA", "NC", "R", "X", "B"]
    
    # Lendo apenas o bloco específico do arquivo
    with open(file_path, 'r') as f:
        # Lógica de extração de bloco (entre 'DLIN' e '9999')
        lines = f.readlines() # Simplificado para o exemplo
        
    pdf = pd.read_fwf(file_path, widths=widths, names=cols, skip_header=1)
    return spark.createDataFrame(pdf)

# Criando View SQL
df_linhas = pwf_to_spark("sistema_ieee30.pwf")
df_linhas.createOrReplaceTempView("v_linhas")

# Query SQL para análise de reatância
res = spark.sql("SELECT * FROM v_linhas WHERE X > 0.05")


In [ ]:
# Carregando o arquivo .pwf diretamente no Pandapower
net = pc.from_ppc(pc.pwf_to_ppc("sistema_ieee14.pwf"))

# Acessando via POO
print(net.line)  # DataFrame de linhas
print(net.bus)   # DataFrame de barras

# Rodando Fluxo de Potência
pp.runpp(net)


In [ ]:
def df_to_pwf_line(row):
    # Formatação posicional rígida do AnaRede
    return f"{str(row['DE']).rjust(5)}{str(row['PARA']).rjust(5)}{str(row['R']).rjust(10)}"

# Usando Pandas para gerar o texto do arquivo
pdf_final = res.toPandas()
lines_pwf = pdf_final.apply(df_to_pwf_line, axis=1)

with open("saida_anaRede.pwf", "w") as f:
    f.write("DLIN\n")
    f.write("\n".join(lines_pwf))
    f.write("\n9999")
